In [68]:
# Data manipulation

import numpy as np
import pandas as pd
pd.options.display.precision = 4
pd.options.mode.chained_assignment = None  

# Machine learning pipeline
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn import set_config
set_config(display="diagram")

In [69]:
df = pd.read_csv("train.csv")

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [71]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [72]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [73]:
#Удаление признаков, которые не влиют  на выживаемость пассажиров.

df_new = df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])            

In [74]:
SEED = 42
TARGET = "Survived"
FEATURES = df_new.columns.drop(TARGET)

NUMERICAL = df_new[FEATURES].select_dtypes('number').columns
print(f"Numerical features: {', '.join(NUMERICAL)}")

CATEGORICAL = pd.Index(np.setdiff1d(FEATURES, NUMERICAL))
print(f"Categorical features: {', '.join(CATEGORICAL)}")

Numerical features: Pclass, Age, SibSp, Parch, Fare
Categorical features: Embarked, Sex


In [75]:
X_train, X_test, y_train, y_test = train_test_split(df_new.drop(columns=TARGET), df[TARGET], 
                                                    test_size=.2, random_state=SEED, 
                                                    stratify=df[TARGET])
num_imputer = SimpleImputer(strategy='mean')
train_num_imputed = num_imputer.fit_transform(X_train[NUMERICAL])

scaler = MinMaxScaler()
train_num_scaled = scaler.fit_transform(train_num_imputed)

cat_imputer = SimpleImputer(strategy='constant', fill_value='missing')
train_cat_imputed = cat_imputer.fit_transform(X_train[CATEGORICAL])

encoder = OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False)
train_cat_encoded = encoder.fit_transform(train_cat_imputed)

train_preprocessed = np.concatenate((train_num_scaled, train_cat_encoded), axis=1)

columns = np.append(NUMERICAL, encoder.get_feature_names_out(CATEGORICAL))
pd.DataFrame(train_preprocessed, columns=columns, index=X_train.index).head()

c:\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,Pclass,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S,Embarked_missing,Sex_male
692,1.0,0.3693,0.000,0.0000,0.1103,0.0,1.0,0.0,1.0
481,0.5,0.3693,0.000,0.0000,0.0000,0.0,1.0,0.0,1.0
527,0.0,0.3693,0.000,0.0000,0.4329,0.0,1.0,0.0,1.0
855,1.0,0.2209,0.000,0.1667,0.0182,0.0,1.0,0.0,0.0
801,0.5,0.3843,0.125,0.1667,0.0512,0.0,1.0,0.0,0.0


In [76]:
model = LogisticRegression()
model.fit(train_preprocessed, y_train)

LogisticRegression()

In [77]:
def calculate_roc_auc(model_pipe, X, y):

    y_proba = model_pipe.predict_proba(X)[:,1]
    return roc_auc_score(y, y_proba)


In [78]:
test_num_imputed = num_imputer.transform(X_test[NUMERICAL])
test_num_scaled = scaler.transform(test_num_imputed)
test_cat_imputed = cat_imputer.transform(X_test[CATEGORICAL])
test_cat_encoded = encoder.transform(test_cat_imputed)
test_preprocessed = np.concatenate((test_num_scaled, test_cat_encoded), axis=1)

print(f"Train ROC-AUC: {calculate_roc_auc(model, train_preprocessed, y_train):.4f}")
print(f"Test ROC-AUC: {calculate_roc_auc(model, test_preprocessed, y_test):.4f}")

Train ROC-AUC: 0.8580
Test ROC-AUC: 0.8403


### Подход № 1

In [79]:
numerical_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())
])

categorical_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('encoder', OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False))
])

preprocessors = ColumnTransformer(transformers=[
    ('num', numerical_pipe, NUMERICAL),
    ('cat', categorical_pipe, CATEGORICAL)
])

pipe = Pipeline([
    ('preprocessors', preprocessors),
    ('model', LogisticRegression())
])

pipe.fit(X_train, y_train)

c:\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('preprocessors',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('encoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  Index(['Embarked', 'Sex'], dtype='object'))])),
                ('model', LogisticRegression())])

In [80]:
print(f"Train ROC-AUC: {calculate_roc_auc(pipe, X_train, y_train):.4f}")
print(f"Test ROC-AUC: {calculate_roc_auc(pipe, X_test, y_test):.4f}")

Train ROC-AUC: 0.8580
Test ROC-AUC: 0.8403


### Подход № 2 

In [81]:
class Imputer(BaseEstimator, TransformerMixin):
    def __init__(self, features, method='constant', value='missing'):
        self.features = features
        self.method = method
        self.value = value
    
    def fit(self, X, y=None):
        if self.method=='mean':
            self.value = X[self.features].mean()
        return self
    
    def transform(self, X):
        X_transformed = X.copy()
        X_transformed[self.features] = X[self.features].fillna(self.value)
        return X_transformed
    
class Scaler(BaseEstimator, TransformerMixin):
    def __init__(self, features):
        self.features = features
    
    def fit(self, X, y=None):
        self.min = X[self.features].min()
        self.range = X[self.features].max()-self.min
        return self
    
    def transform(self, X):
        X_transformed = X.copy()
        X_transformed[self.features] = (X[self.features]-self.min)/self.range
        return X_transformed
  
class Encoder(BaseEstimator, TransformerMixin):
    def __init__(self, features, drop='first'):
        self.features = features
        self.drop = drop
    
    def fit(self, X, y=None):
        self.encoder = OneHotEncoder(sparse=False, drop=self.drop)
        self.encoder.fit(X[self.features])
        return self
    
    def transform(self, X):
        X_transformed = pd.concat([X.drop(columns=self.features).reset_index(drop=True), 
                                   pd.DataFrame(self.encoder.transform(X[self.features]), 
                                                columns=self.encoder.get_feature_names_out(self.features))],
                                  axis=1)
        return X_transformed
        
pipe = Pipeline([
    ('num_imputer', Imputer(NUMERICAL, method='mean')),
    ('scaler', Scaler(NUMERICAL)),
    ('cat_imputer', Imputer(CATEGORICAL)),
    ('encoder', Encoder(CATEGORICAL)),
    ('model', LogisticRegression())
])

pipe.fit(X_train, y_train)   

c:\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('num_imputer',
                 Imputer(features=Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object'),
                         method='mean',
                         value=Pclass     2.3090
Age       29.8077
SibSp      0.4930
Parch      0.3904
Fare      31.8198
dtype: float64)),
                ('scaler',
                 Scaler(features=Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object'))),
                ('cat_imputer',
                 Imputer(features=Index(['Embarked', 'Sex'], dtype='object'))),
                ('encoder',
                 Encoder(features=Index(['Embarked', 'Sex'], dtype='object'))),
                ('model', LogisticRegression())])

In [82]:
print(f"Train ROC-AUC: {calculate_roc_auc(pipe, X_train, y_train):.4f}")
print(f"Test ROC-AUC: {calculate_roc_auc(pipe, X_test, y_test):.4f}")

Train ROC-AUC: 0.8580
Test ROC-AUC: 0.8403


In [83]:
# Подбор гиперпараметров

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

class LogisticRegressionTuner(BaseEstimator, TransformerMixin):
    def __init__(self, param_grid=None, cv=3):
        self.param_grid = param_grid
        self.cv = cv
    
    def fit(self, X, y=None):
        self.model = LogisticRegression()
        self.grid_search = GridSearchCV(self.model, self.param_grid, cv=self.cv)
        self.grid_search.fit(X, y)
        return self
    
    def transform(self, X):
        # LogisticRegressionTuner не изменяет данные, поэтому просто возвращаем X
        return X
    
    # Определите сетку параметров, которые вы хотите оптимизировать для LogisticRegression
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l2']
}

# Добавьте LogisticRegressionTuner в ваш конвейер
pipe = Pipeline([
    ('num_imputer', Imputer(NUMERICAL, method='mean')),
    ('scaler', Scaler(NUMERICAL)),
    ('cat_imputer', Imputer(CATEGORICAL)),
    ('encoder', Encoder(CATEGORICAL)),
    ('logistic_regression_tuner', LogisticRegressionTuner(param_grid=param_grid, cv=3)),
    ('model', LogisticRegression())  # Добавьте оптимизированную модель
])

# Запустите конвейер на тренировочных данных
pipe.fit(X_train, y_train)

c:\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('num_imputer',
                 Imputer(features=Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object'),
                         method='mean',
                         value=Pclass     2.3090
Age       29.8077
SibSp      0.4930
Parch      0.3904
Fare      31.8198
dtype: float64)),
                ('scaler',
                 Scaler(features=Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object'))),
                ('cat_imputer',
                 Imputer(features=Index(['Embarked', 'Sex'], dtype='object'))),
                ('encoder',
                 Encoder(features=Index(['Embarked', 'Sex'], dtype='object'))),
                ('logistic_regression_tuner',
                 LogisticRegressionTuner(param_grid={'C': [0.001, 0.01, 0.1, 1,
                                                           10, 100],
                                                     'penalty': ['l2']})),
                ('model', LogisticRegression())])

In [84]:
print(f"Train ROC-AUC: {calculate_roc_auc(pipe, X_train, y_train):.4f}")
print(f"Test ROC-AUC: {calculate_roc_auc(pipe, X_test, y_test):.4f}")

Train ROC-AUC: 0.8580
Test ROC-AUC: 0.8403


Конвейер для обучения нейронной сети с использованием TensorFlow

In [85]:
import tensorflow as tf
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin

class NeuralNetworkTrainer(BaseEstimator, TransformerMixin):
    def __init__(self, epochs=10, batch_size=32, verbose=1):
        self.epochs = epochs
        self.batch_size = batch_size
        self.verbose = verbose
        self.model = None
    
    def fit(self, X, y=None):
        input_shape = (X.shape[1],)  # Определите форму входных данных на основе X
        num_classes = len(np.unique(y))  # Количество классов в задаче классификации
        
        model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(2048, activation='relu', input_shape=input_shape),
            tf.keras.layers.Dense(num_classes, activation='softmax')
        ])

        model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        
        model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size, verbose=self.verbose)
        self.model = model
        return self
    
    def transform(self, X):
        # NeuralNetworkTrainer не изменяет данные, поэтому просто возвращаем X
        return X

    def predict(self, X):
        # Предсказать метки классов для новых данных X
        if self.model is None:
            raise ValueError("Модель не обучена")
        y_pred = self.model.predict(X)
        return np.argmax(y_pred, axis=1)
    def score(self, X, y=None):
        # Оцените производительность модели на данных X и y
        if self.model is None:
            raise ValueError("Модель не обучена")
        y_pred = self.model.predict(X)
        accuracy = accuracy_score(y, np.argmax(y_pred, axis=1))
        return accuracy

# Добавьте NeuralNetworkTrainer в ваш конвейер
pipe = Pipeline([
    ('num_imputer', Imputer(NUMERICAL, method='mean')),
    ('scaler', Scaler(NUMERICAL)),
    ('cat_imputer', Imputer(CATEGORICAL)),
    ('encoder', Encoder(CATEGORICAL)),
    ('neural_network_trainer', NeuralNetworkTrainer()),
])

# Запустите конвейер на тренировочных данных
pipe.fit(X_train, y_train)

Epoch 1/10


c:\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


23/23 [==============================] - 1s 2ms/step - loss: 0.5703 - accuracy: 0.6812
Epoch 2/10
23/23 [==============================] - 0s 2ms/step - loss: 0.4783 - accuracy: 0.7865
Epoch 3/10
23/23 [==============================] - 0s 2ms/step - loss: 0.4788 - accuracy: 0.7963
Epoch 4/10
23/23 [==============================] - 0s 2ms/step - loss: 0.4521 - accuracy: 0.7935
Epoch 5/10
23/23 [==============================] - 0s 2ms/step - loss: 0.4395 - accuracy: 0.7992
Epoch 6/10
23/23 [==============================] - 0s 2ms/step - loss: 0.4344 - accuracy: 0.8146
Epoch 7/10
23/23 [==============================] - 0s 2ms/step - loss: 0.4304 - accuracy: 0.8174
Epoch 8/10
23/23 [==============================] - 0s 2ms/step - loss: 0.4290 - accuracy: 0.8090
Epoch 9/10
23/23 [==============================] - 0s 2ms/step - loss: 0.4219 - accuracy: 0.8174
Epoch 10/10
23/23 [==============================] - 0s 2ms/step - loss: 0.4204 - accuracy: 0.8132


Pipeline(steps=[('num_imputer',
                 Imputer(features=Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object'),
                         method='mean',
                         value=Pclass     2.3090
Age       29.8077
SibSp      0.4930
Parch      0.3904
Fare      31.8198
dtype: float64)),
                ('scaler',
                 Scaler(features=Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object'))),
                ('cat_imputer',
                 Imputer(features=Index(['Embarked', 'Sex'], dtype='object'))),
                ('encoder',
                 Encoder(features=Index(['Embarked', 'Sex'], dtype='object'))),
                ('neural_network_trainer', NeuralNetworkTrainer())])

In [86]:
#  Предсказание на тестовых данных
y_pred = pipe.predict(X_test)

6/6 [==============================] - 0s 1ms/step


In [87]:
from sklearn.metrics import accuracy_score

# Вычислите оценку точности (accuracy) модели
accuracy = accuracy_score(y_test, y_pred)

# Выведите оценку на экран
print("Accuracy:", accuracy)


Accuracy: 0.8044692737430168


In [88]:
import tensorflow as tf
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

class NeuralNetworkTrainer(BaseEstimator, TransformerMixin):
    def __init__(self, epochs=10, batch_size=32, verbose=1):
        self.epochs = epochs
        self.batch_size = batch_size
        self.verbose = verbose
        self.model = None
        
    def fit(self, X, y=None):
        input_shape = (X.shape[1],)  # Определите форму входных данных на основе X
        num_classes = len(np.unique(y))  # Количество классов в задаче классификации
        
        model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(2048, activation='relu', input_shape=input_shape),
            tf.keras.layers.Dense(num_classes, activation='softmax')
        ])
        
        model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        
        model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size, verbose=self.verbose)
        self.model = model
        return self
    
    def transform(self, X):
        # NeuralNetworkTrainer не изменяет данные, поэтому просто возвращаем X
        return X

    def predict(self, X):
        # Предсказать метки классов для новых данных X
        if self.model is None:
            raise ValueError("Модель не обучена")
        y_pred = self.model.predict(X)
        return np.argmax(y_pred, axis=1)
        
    def score(self, X, y=None):
        # Оцените производительность модели на данных X и y
        if self.model is None:
            raise ValueError("Модель не обучена")
        y_pred = self.model.predict(X)
        accuracy = accuracy_score(y, np.argmax(y_pred, axis=1))
        return accuracy
    
    # Остальной код остается без изменений

# Создайте экземпляр нейронной сети
neural_network = NeuralNetworkTrainer()

# Определите сетку параметров для нейронной сети
param_grid = {
    'neural_network__epochs': [10, 20, 30],  # Разные значения числа эпох
    'neural_network__batch_size': [32, 64, 128],  # Разные значения размера пакета
    'neural_network__verbose': [0, 1],  # Разные уровни вывода
}

# Создайте конвейер с нейронной сетью
pipeline = Pipeline([
    ('num_imputer', Imputer(NUMERICAL, method='mean')),
    ('scaler', Scaler(NUMERICAL)),
    ('cat_imputer', Imputer(CATEGORICAL)),
    ('encoder', Encoder(CATEGORICAL)),
    ('neural_network', neural_network),
])
# Создайте GridSearchCV объект с конвейером и сеткой параметров
grid_search = GridSearchCV(pipeline, param_grid, cv=3, n_jobs=-1)

# Запустите поиск по сетке на тренировочных данных
grid_search.fit(X_train, y_train)

# Получите наилучшие параметры
best_params = grid_search.best_params_
print("Наилучшие параметры:", best_params)

c:\Python311\Lib\site-packages\sklearn\model_selection\_search.py:976: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
c:\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Наилучшие параметры: {'neural_network__batch_size': 32, 'neural_network__epochs': 10, 'neural_network__verbose': 0}


In [89]:
# Сделайте предсказание на тестовых данных
y_pred = pipe.predict(X_test)

6/6 [==============================] - 0s 3ms/step


In [90]:
from sklearn.metrics import accuracy_score

# Вычислите оценку точности (accuracy) модели
accuracy = accuracy_score(y_test, y_pred)

# Выведите оценку на экран
print("Accuracy:", accuracy)

Accuracy: 0.8044692737430168
